In [2]:
# Install required libraries
!pip install gradio transformers torch gtts moviepy openai-whisper --quiet

# Import necessary libraries
import gradio as gr
import moviepy.editor as mp
import os
from gtts import gTTS
import whisper
from transformers import pipeline

# Load Whisper model (this may take a moment)
whisper_model = whisper.load_model("base")

# Define translation models for South Indian languages and their gTTS language codes
translation_models = {
    "English to Tamil": {"model": "Helsinki-NLP/opus-mt-en-ta", "tts_lang": "ta"},
    "English to Telugu": {"model": "Helsinki-NLP/opus-mt-en-te", "tts_lang": "te"},
    "English to Kannada": {"model": "Helsinki-NLP/opus-mt-en-kn", "tts_lang": "kn"},
    "English to Malayalam": {"model": "Helsinki-NLP/opus-mt-en-ml", "tts_lang": "ml"}
}

def process_video(video_file, target_language):
    # Save the uploaded video file
    input_video_path = "input_video.mp4"
    with open(input_video_path, "wb") as f:
        f.write(video_file.read())

    # Load the video using MoviePy and extract its audio
    video_clip = mp.VideoFileClip(input_video_path)
    audio_path = "extracted_audio.wav"
    video_clip.audio.write_audiofile(audio_path, logger=None)

    # Transcribe the audio using Whisper
    transcription_result = whisper_model.transcribe(audio_path)
    original_text = transcription_result["text"]

    # Set up the translation pipeline using the selected model
    model_name = translation_models[target_language]["model"]
    translator = pipeline("translation", model=model_name)
    translation = translator(original_text)[0]["translation_text"]

    # Generate new speech (TTS) for the translated text using gTTS
    tts_language = translation_models[target_language]["tts_lang"]
    tts = gTTS(text=translation, lang=tts_language)
    tts_audio_path = "translated_audio.mp3"
    tts.save(tts_audio_path)

    # Load the generated TTS audio and set it as the audio for the original video
    new_audio = mp.AudioFileClip(tts_audio_path)
    # Note: The new audio length may differ from the original. Here we set the new audio and cut or loop as needed.
    # For simplicity, we use the new audio as-is.
    final_video = video_clip.set_audio(new_audio)
    output_video_path = "translated_video.mp4"
    final_video.write_videofile(output_video_path, codec="libx264", audio_codec="aac", logger=None)

    # Clean up temporary files if needed
    # os.remove(input_video_path)
    # os.remove(audio_path)
    # os.remove(tts_audio_path)

    return output_video_path

# Create a Gradio interface
iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Video(label="Input Video (English)"),
        gr.Dropdown(choices=list(translation_models.keys()), label="Select Target Language")
    ],
    outputs=gr.Video(label="Translated Video"),
    title="Video Translator with TTS",
    description=("Upload a video with English speech. The system will transcribe, translate to a South Indian language, "
                 "generate new speech for the translation, and output a video with the new audio track.")
)

# Launch the Gradio app
iface.launch(share=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/

  if event.key is 'enter':

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 110MiB/s]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4aa8e6242e38988c20.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
